# CZ gate  without Taylor

In [ ]:
import sys
sys.path.append("../")
from system import *

s1 =  system('x-o-',MMA=True) 
#s1 = load_object('2')

Initializing system x-o-  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...


# $\Delta_{E\gamma} = \ \dfrac{\Delta_E}{\gamma}\, C^n \qquad \Delta_{e\gamma} = \ \dfrac{\Delta_e}{\gamma}\, C^m$

In [53]:
var('n','m',domain='positive')
n = n
m = m

# Effective Hamiltonian Matrix obtain NO $C \longrightarrow \infty$ limit

In [54]:
phi_val = 0
var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g','phi',domain='positive')
var('DEt',domain='complex',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='complex',  latex_name =r'\tilde{\Delta }_e')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')

eff_hamiltonian_tayl = sg.copy( s1.eff_hamiltonian.parent().zero())
for diag in s1.pos_gs:
    print(f'Calculating {diag} out of {s1.pos_gs}...')
    #Take only one of the compl conj terms. Switch maybe second 0 to 1
    min_expr_len = 10**10
    for term in range(1):
        b = s1.eff_hamiltonian[diag,diag]

        b = b.operands()[1].operands()[term]
        #b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
        #b = b.subs(De=+0.5*I*gamma   +Det*gamma)
        b = b.subs(De=Deg*gamma*C^m)
        b = b.subs(DE=DEg*gamma*C^n)
        
        
        var('R_f',domain='positive')#ratio  (g_f/g)^2
        b = b.subs(g_f=g*sqrt(R_f))

        var('R_v',domain='positive',  latex_name =r'R_{\nu}') #ratio (v/g)^2
        b = b.subs(v=g*sqrt(R_v))

        var('r_b',domain='positive')
        b = b.subs(kappa_b=gamma/r_b)
        
        var('r_f',domain='positive',latex_name =r'r_f')
        b = b.subs(gamma_f=gamma*r_f)
        
        var('r_g',domain='positive',latex_name =r'r_g')
        b = b.subs(gamma_g=gamma*r_g)
        
        b = b.subs(g = sqrt(C*gamma*kappa_c)) 
        b = b.subs(r_b = c/(C*R_v))

        b = b*gamma  #Omega has to be substituted with Omega/gamma
        
        b = b.subs(c=np.pi,R_f=1,Deg=np.e,DEg=np.sqrt(5),r_g=np.sqrt(3),r_f=np.sqrt(2), phi=phi_val) 

        b = b._mathematica_().Factor()._sage_()
        b = MMA_simplify(b)#,full=True)
        #holds the smallest expression
        if len(str(b))<min_expr_len:
            min_expr_len = len(str(b))
            h_eff = b
        
        h_eff = SR(str(h_eff).replace('Sqrt','sqrt'))
    '''
    q = h_eff.subs(C=1/x)
    q = MMA_simplify(q)
    tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
    tayl = tayl.subs(x=1/C)
    '''
    tayl = h_eff
    
    #tayl =  (tayl + conjugate(tayl))/2      # try with + conjugate_transp

    #taylr = taylr.expand()
    tayl = MMA_simplify(tayl,full=False)
    tayl = SR(str(tayl).replace('Sqrt','sqrt'))
    eff_hamiltonian_tayl[diag,diag] = Omega^2/gamma * tayl

print('\nReal part of H_eff in ground state subspace taylor expanded in 1/C: ')
show(symround(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs],digits=4))

eff_hamiltonian_tayl = (eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs] + conjugate(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs]))/2

Calculating 0 out of [0, 1]...
Calculating 1 out of [0, 1]...

Real part of H_eff in ground state subspace taylor expanded in 1/C: 


[                                                                                                                     0.9631*I*Omega^2/((2.154*I*C^n + C + 1.515)*gamma)                                                                                                                                                                       0]
[                                                                                                                                                                      0 (-0.9631*I*C^m + 0.3679*C + 0.1772)*Omega^2/((0.05423*I*C^2 - 2.154*I*C^(m + n) + C^(m + 1) + 1.515*C^m + 0.8226*C^(n + 1) + 0.7627*I*C + 0.3961*C^n + 0.2787*I)*gamma)]

# CZ gate time

In [57]:
#l_0-l_1-l_2+l_3
denom = eff_hamiltonian_tayl[0,0]+eff_hamiltonian_tayl[3,3]-eff_hamiltonian_tayl[1,1]-eff_hamiltonian_tayl[2,2]

t_g = np.pi/denom
t_g = MMA_simplify(t_g,full=False) #not full if too slow
t_g = SR(str(t_g).replace('Sqrt','sqrt')).simplify()

In [58]:
show(symround((t_g)))
#GHZ REGIME 
rf =1 
de=122183415123789/100000000000000
r_v=1.01014311134038/c
#show(symround((t_g.subs(R_f =rf ,DEg=de,R_v=r_v))))

-6.3*gamma/(Omega^2*((0.96*I*C^m + 0.37*C + 0.18)/(-0.054*I*C^2 + 2.2*I*C^(m + n) + C^(m + 1) + 1.5*C^m + 0.82*C^(n + 1) - 0.77*I*C + 0.39*C^n - 0.28*I) + (-0.96*I*C^m + 0.37*C + 0.18)/(0.054*I*C^2 - 2.2*I*C^(m + n) + C^(m + 1) + 1.5*C^m + 0.82*C^(n + 1) + 0.77*I*C + 0.39*C^n + 0.28*I) - 0.96*I/(2.2*I*C^n + C + 1.5) + 0.96*I/(-2.2*I*C^n + C + 1.5)))

In [46]:
q = t_g.subs(C=1/x)
q = MMA_simplify(q)
tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
tayl = tayl.subs(x=1/C)
show(symround(tayl))

6.3*gamma/(Omega^2*((3.7*I*C^(2*m) - I*C - 5.4*C^(m + 1) - 1.4*C^m - 0.12*I)/(-5.8*C^(2*m) + 8.2*I*C^(2*m + n) + 1.8*C - 7.4*C^(2*m + 1) - 12.*C^(m + n + 1) - 3.0*C^(m + n) - 11.*I*C^(m + 1) - 2.1*I*C^m - 2.2*I*C^(n + 1) - 0.28*I*C^n + 0.20) + (-3.7*I*C^(2*m) + I*C - 5.4*C^(m + 1) - 1.4*C^m + 0.12*I)/(-5.8*C^(2*m) - 8.2*I*C^(2*m + n) + 1.8*C - 7.4*C^(2*m + 1) - 12.*C^(m + n + 1) - 3.0*C^(m + n) + 11.*I*C^(m + 1) + 2.1*I*C^m + 2.2*I*C^(n + 1) + 0.28*I*C^n + 0.20) - (0.89*C^m + 0.66*I*C + 0.16*I)/((C^(m + n - 1) + 0.70*I*C^(m - 1) + 0.89*I*C^m + 0.18*I*C^(n - 1) + 0.73*I*C^n - 0.13/C - 0.68)*C) - (0.89*C^m - 0.66*I*C - 0.16*I)/((C^(m + n - 1) - 0.70*I*C^(m - 1) - 0.89*I*C^m - 0.18*I*C^(n - 1) - 0.73*I*C^n - 0.13/C - 0.68)*C) + 0.50*I/(1.1*I*C^n + C + 0.79) - 0.50*I/(-1.1*I*C^n + C + 0.79)))

# Lindblau Operators no $C \longrightarrow \infty$ limit

Choose which of the Lindblau operators to view and which non zero element of it

In [59]:
lind_op_number = len(s1.lindblau_list)

eff_lind_taylor = []
eff_lind = []
eff_lind_coeff = []
for lind_op in range(lind_op_number):
    print(f'{lind_op} out of {lind_op_number-1}')
    eff_lind_taylor.append([])
    eff_lind.append([])
    L_matrix = s1.eff_lindblau_list[lind_op]
    L_nonzeros = []
    L_nonzeros_pos = []
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))
    eff_lind_coeff.append(s1.L_coeffs[lind_op])
    for which in range(len(L_nonzeros)):

        L_elem = L_nonzeros[which]

        #L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
        #L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
        L_elem = L_elem.subs(De=Deg*gamma*C^m)
        L_elem = L_elem.subs(DE=DEg*gamma*C^n)

        #COUPLINGS

        L_elem = L_elem.subs(g_f=g*sqrt(R_f))

        L_elem = L_elem.subs(v=g*sqrt(R_v))

        #ABSORBTIONS
        L_elem = L_elem.subs(kappa_b=gamma/r_b)

        L_elem = L_elem.subs(gamma_g=gamma*r_g)

        L_elem = L_elem.subs(gamma_f=gamma*r_f)

        L_elem = L_elem.subs(g = sqrt(C*gamma*kappa_c)) 
        L_elem = L_elem.subs(r_b = c/(C*R_v))
        
        L_elem = L_elem.subs(c=np.pi,R_f=1,Deg=np.e,DEg=np.sqrt(5),r_g=np.sqrt(3),r_f=np.sqrt(2),phi=phi_val)
        
        L_elem = L_elem._mathematica_().Factor()._sage_()


        L_elem = MMA_simplify(L_elem)#,full=True)

        L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))
        
        L_elem_show = symround(L_elem)
        
        eff_lind[lind_op].append(L_elem_show)
        
        var('x')
        '''
        q = L_elem.subs(C=1/x)
        q = MMA_simplify(q)
        tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
        tayl = tayl.subs(x=1/C)
        '''
        tayl = L_elem
        tayl = SR(str(tayl).replace('Sqrt','sqrt'))
        tayl_show = symround(tayl)
        
        eff_lind_taylor[-1].append(tayl_show)

0 out of 5
1 out of 5
2 out of 5
3 out of 5
4 out of 5
5 out of 5


# See results

In [61]:
lindop = 0
#t_g = gamma/Omega^2 * C^0.5
a = eff_lind_taylor[lindop][0]
err = t_g*a*conjugate(a)
err = SR(str(MMA_simplify( err,full=False )).replace('Sqrt','sqrt')).simplify().subs(m=1/2,n=1/2)
show( symround( err ) )
print(f'coeff:')
show(eff_lind_coeff[lindop])

-6.3*C/((C + 2.2*I*sqrt(C) + 1.5)*(C - 2.2*I*sqrt(C) + 1.5)*((0.37*C + 0.96*I*sqrt(C) + 0.18)/(-0.054*I*C^2 + 1.8*C^1.5 + 1.4*I*C + 1.9*sqrt(C) - 0.28*I) + (0.37*C - 0.96*I*sqrt(C) + 0.18)/(0.054*I*C^2 + 1.8*C^1.5 - 1.4*I*C + 1.9*sqrt(C) + 0.28*I) - 0.96*I/(C + 2.2*I*sqrt(C) + 1.5) + 0.96*I/(C - 2.2*I*sqrt(C) + 1.5)))

coeff:


sqrt(kappa_c)

In [40]:
show(err.numerator().expand())

-25.0250910782765*C^10 - (3.552713678800501e-15*I)*C^(19/2) - 118.64654534562099*C^9 + (1.4210854715202004e-14*I)*C^(17/2) - 184.3306254407749*C^8 + (3.552713678800501e-14*I)*C^(15/2) - 134.39734880720943*C^7 + (3.730349362740526e-14*I)*C^(13/2) - 51.97685558157963*C^6 + (1.4654943925052066e-14*I)*C^(11/2) - 10.913747313380627*C^5 + (2.4841240175987878e-15*I)*C^(9/2) - 1.1802487076042534*C^4 + (6.765421556309548e-17*I)*C^(7/2) - 0.05753729972755995*C^3 + (4.87890977618477e-18*I)*C^(5/2) - 0.001247783406923433*C^2 + (5.082197683525802e-21*I)*C^(3/2) - (9.874985575007444e-06)*C

In [41]:
show(err.denominator().expand())

1.0*C^(21/2) + (1.10546764698699e-16*I)*C^10 + 3.612255692681547*C^(19/2) + (7.565627119140737e-16*I)*C^9 + 3.174545228184853*C^(17/2) + (3.0950859755482263e-15*I)*C^8 + 1.6958427083798973*C^(15/2) + (6.189725079065766e-15*I)*C^7 + 0.7640043117320927*C^(13/2) + (3.726914035923889e-15*I)*C^6 + 0.21942216982783574*C^(11/2) + (8.374199850800896e-16*I)*C^5 + 0.027130885467337672*C^(9/2) - (8.895138788674788e-18*I)*C^4 + 0.0006942663563354894*C^(7/2) - (1.477971113921486e-17*I)*C^3 - (6.169866841524388e-17)*C^(5/2) + (4.608117169979151e-19*I)*C^2 - (1.537658830000464e-18)*C^(3/2) - (1.7405075117658746e-20*I)*C - (9.190481095191023e-21)*sqrt(C) - 1.312925870741571e-22*I